In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm, linear_model
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler,PolynomialFeatures, LabelEncoder,FunctionTransformer
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from scipy import stats
%matplotlib inline
titanic = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

/home/nerd/Data/Conda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:

titanic['FamilyCount']= (titanic['SibSp'])+(titanic['Parch'])
titanic['IsFemale'] = (titanic.Sex=='female').astype(int)
titanic['IsMale'] = (titanic.Sex=='male').astype(int)
testdf['FamilyCount']= (testdf['SibSp'])+(testdf['Parch'])
testdf['IsFemale'] = (testdf.Sex=='female').astype(int)
testdf['IsMale'] = (testdf.Sex=='male').astype(int)
titanic['Cabin'] =titanic['Cabin'].fillna(value='?')
titanic['Embarked'] =titanic['Embarked'].fillna(value='S')
titanic['Age'] =titanic['Age'].fillna(value='?')
testdf['Cabin'] =testdf['Cabin'].fillna(value='?')
testdf['Embarked'] =testdf['Embarked'].fillna(value='S')
testdf['Age'] =testdf['Age'].fillna(value='?')
testdf.loc[152,'Fare']=13.6755 # we fill in the missing fare from the test data here. 

In [3]:
titanic['IsAlone']= (titanic['FamilyCount']==0).astype(int)
testdf['IsAlone']= (testdf['FamilyCount']==0).astype(int)

In [4]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilyCount,IsFemale,IsMale,IsAlone
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,?,S,1,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,1,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,?,S,0,1,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,1,1,0,0
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,?,S,0,0,1,1


In [5]:
def titlepull(title_string):
    title_list = title_string.split()
    for possible_title in title_list:
        if possible_title[-1] == '.':
            return possible_title
def cabinpull(cabin_string):
    if cabin_string[0]=='?':
        return '?'
    else:
        return cabin_string[0]
    

In [6]:
def ticketpull(ticketstring):
    if ticketstring == 'LINE':
        return 9
    string = ticketstring.split(' ')[-1]
    return string[0]
    

In [7]:
titanic['Deck']=titanic.Cabin.apply(cabinpull)
titanic['Title']= titanic.Name.apply(titlepull)
testdf['Deck']=testdf.Cabin.apply(cabinpull)
testdf['Title']= testdf.Name.apply(titlepull)
titanic['Ticketnum']= titanic.Ticket.apply(ticketpull)
testdf['Ticketnum']=testdf.Ticket.apply(ticketpull)


In [8]:
titanic[titanic['Ticketnum']=='L']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilyCount,IsFemale,IsMale,IsAlone,Deck,Title,Ticketnum


In [9]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilyCount,IsFemale,IsMale,IsAlone,Deck,Title,Ticketnum
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,?,S,1,0,1,0,?,Mr.,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,1,1,0,0,C,Mrs.,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,?,S,0,1,0,1,?,Miss.,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,1,1,0,0,C,Mrs.,1
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,?,S,0,0,1,1,?,Mr.,3


In [10]:
decks_list = ['?','C','B','D','E','F','A','G','T']

In [11]:
name_mappings = {
    'Dona.':'Royalty',
    'Countess.':'Royalty',
    'Sir.':'Royalty',
    'Jonkheer.':'Royalty',
    'Don.':'Royalty',
    'Lady.':'Royalty',
    'Ms.':'Miss.',
    'Col.':'Officer',
    'Capt.':'Officer',
    'Major.':'Officer',
    'Col.':'Officer',
    'Mlle.':'Miss.',
    'Mme.':'Mrs.',
    'Mrs.':'Mrs.',
    'Miss.':'Miss',
    'Mr.':'Mr.',
    'Master.':'Master.',
    'Dr.':'Dr.',
    'Rev.':'Rev.'
    
    
}

In [12]:


titanic['Title']= titanic['Title'].map(name_mappings)
testdf['Title']= testdf['Title'].map(name_mappings)

In [13]:
validtitles = list(set(name_mappings.values()))
titanic.FamilyCount = titanic['FamilyCount'].astype("category", categories=range(11), ordered=False)


In [14]:
testdf.FamilyCount = testdf['FamilyCount'].astype("category", categories=range(11), ordered=False)

In [15]:
titanic.Title = titanic['Title'].astype("category", categories=validtitles, ordered=False)

In [16]:
testdf.Title = testdf['Title'].astype("category", categories=validtitles, ordered=False)

In [17]:
titanic.Deck = titanic['Deck'].astype("category", categories=decks_list, ordered=False)

In [18]:
testdf.Deck = testdf['Deck'].astype("category", categories=decks_list, ordered=False)

In [19]:
def numeric_features(df):
    return df.select_dtypes(exclude= ['object','category'])
numeric_features_tf = FunctionTransformer(numeric_features, validate=False)

In [20]:
numeric_features(titanic)

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,IsFemale,IsMale,IsAlone
0,1,0,3,1,0,7.2500,0,1,0
1,2,1,1,1,0,71.2833,1,0,0
2,3,1,3,0,0,7.9250,1,0,1
3,4,1,1,1,0,53.1000,1,0,0
4,5,0,3,0,0,8.0500,0,1,1
5,6,0,3,0,0,8.4583,0,1,1
6,7,0,1,0,0,51.8625,0,1,1
7,8,0,3,3,1,21.0750,0,1,0
8,9,1,3,0,2,11.1333,1,0,0
9,10,1,2,1,0,30.0708,1,0,0


In [21]:
age_transformers = [('numfeat' ,numeric_features_tf)]

In [22]:
fu_age_impute = FeatureUnion(age_transformers)

In [23]:
knr = KNeighborsRegressor(n_jobs=-1)
lr = LinearRegression()
ss = StandardScaler()

In [24]:
age_impute_pipe_lr =  Pipeline(
    [
        ('fu',fu_age_impute),
        ('ss',ss),
        ('lr',lr)
    ]

)

In [25]:
age_impute_pipe_kn =  Pipeline(
    [
        ('fu_age',fu_age_impute),
        ('ss',ss),
        ('knr',knr)
    ]

)

In [26]:
age_params = {
    
}

In [27]:
gs_age_kn = GridSearchCV(age_impute_pipe_kn, param_grid=age_params, cv=5)
gs_age_lr = GridSearchCV(age_impute_pipe_lr, param_grid=age_params, cv=5)

In [28]:
titanic_ages =titanic[(titanic['Age']!='?')]
titanic_no_ages = titanic[(titanic['Age']=='?')]

test_ages =testdf[(testdf['Age']!='?')]
test_no_ages = testdf[(testdf['Age']=='?')]


In [29]:
X_train = titanic_ages.drop(['Age','Survived'],axis=1)
y_train= titanic_ages['Age']


In [30]:
X_train.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'FamilyCount', 'IsFemale', 'IsMale',
       'IsAlone', 'Deck', 'Title', 'Ticketnum'],
      dtype='object')

In [31]:
#X_test1.columns

In [32]:
#X_train.shape

In [33]:
#X_test1.shape

In [34]:
X_test1 =titanic_no_ages.drop(['Age','Survived'], axis=1)
X_test2 = test_no_ages.drop('Age', axis=1)

In [35]:
X_test1.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'FamilyCount', 'IsFemale', 'IsMale',
       'IsAlone', 'Deck', 'Title', 'Ticketnum'],
      dtype='object')

In [36]:
X_test1.shape

(177, 17)

In [37]:
X_test1.head()

,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilyCount,IsFemale,IsMale,IsAlone,Deck,Title,Ticketnum
5,6,3,"Moran, Mr. James",male,0,0,330877,8.4583,?,Q,0,0,1,1,?,Mr.,3
17,18,2,"Williams, Mr. Charles Eugene",male,0,0,244373,13.0000,?,S,0,0,1,1,?,Mr.,2
19,20,3,"Masselmani, Mrs. Fatima",female,0,0,2649,7.2250,?,C,0,1,0,1,?,Mrs.,2
26,27,3,"Emir, Mr. Farred Chehab",male,0,0,2631,7.2250,?,C,0,0,1,1,?,Mr.,2
28,29,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,0,0,330959,7.8792,?,Q,0,1,0,1,?,Miss,3


In [38]:
X_test2.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'FamilyCount', 'IsFemale', 'IsMale',
       'IsAlone', 'Deck', 'Title', 'Ticketnum'],
      dtype='object')

In [39]:
X_test2.shape

(86, 17)

In [40]:
gs_age_lr.fit(X_train,y_train)
print(gs_age_lr.best_score_)

0.189979354111


In [41]:
gs_age_kn.fit(X_train,y_train)
print(gs_age_kn.best_score_)

0.137032623613


In [42]:
age_impute_pipe_kn.fit(X_train,y_train)

Pipeline(steps=[('fu_age', FeatureUnion(n_jobs=1,
       transformer_list=[('numfeat', FunctionTransformer(accept_sparse=False,
          func=<function numeric_features at 0x7f274ef382f0>,
          inv_kw_args=None, inverse_func=None, kw_args=None, pass_y=False,
          validate=False))],
       transfor...kowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='uniform'))])

In [43]:
age_impute_pipe_kn.score(X_train,y_train)

0.47444247900525049

In [44]:
age_impute_pipe_kn.predict(X_test1)

array([25.1, 38.4, 20.8, 25.1, 20.8, 25.1, 34.0, 20.8, 22.5, 22.3, 22.3,
       24.4, 20.8, 25.4, 44.2, 45.6, 24.0, 23.8, 23.8, 23.0, 30.8, 33.0,
       35.6, 28.4, 24.7, 34.1, 38.5, 20.0, 20.4, 31.1, 31.1, 8.8, 28.2,
       54.6, 6.0, 7.0, 34.4, 46.4, 24.9, 32.3, 21.2, 8.8, 25.0, 27.7, 9.95,
       27.0, 24.9, 24.9, 24.7, 28.4, 32.8, 28.6, 47.5, 28.4, 32.4, 47.5,
       47.5, 47.5, 28.4, 26.2, 33.6, 23.9, 33.2, 7.6, 12.75, 24.6, 39.4,
       29.4, 40.9, 31.6, 24.6, 24.6, 24.2, 24.6, 24.6, 42.0, 22.6, 26.6,
       3.7, 36.2, 32.6, 33.6, 31.6, 29.4, 28.6, 28.6, 25.2, 26.4, 25.0,
       31.8, 46.0, 32.2, 32.2, 38.8, 32.2, 36.4, 54.4, 38.8, 3.7, 29.0,
       28.4, 28.4, 34.6, 48.2, 24.8, 23.6, 31.5, 31.5, 31.4, 31.5, 21.0,
       30.5, 43.6, 25.8, 31.4, 25.8, 25.8, 34.6, 27.8, 26.6, 24.6, 30.4,
       36.0, 11.4, 38.2, 35.6, 35.6, 42.4, 31.2, 24.6, 31.2, 30.0, 42.6,
       25.2, 23.6, 26.4, 27.2, 22.3, 31.8, 32.3, 37.8, 40.2, 18.9, 27.3,
       20.5, 22.0, 42.6, 24.3, 16.9, 29.6, 27.6, 27

In [45]:
X_test1.shape

(177, 17)

In [46]:
X_test1['Age'] = age_impute_pipe_kn.predict(X_test1)

In [47]:
X_test2['Age']= age_impute_pipe_kn.predict(X_test2)

In [48]:
titanic.loc[X_test1.index,'Age'] = X_test1['Age']
testdf.loc[X_test2.index,'Age'] = X_test2['Age']

In [49]:
def ageband(age):
    ageband= (int(age)//13)+1
    return ageband

In [50]:
titanic['AgeBand']=titanic.Age.apply(ageband)
testdf['AgeBand']=testdf.Age.apply(ageband)

In [51]:
def findsurname(wife):
    if wife[-1]==')':
        return wife.split()[0]
    
def findhusband(wife, husband):
    for num in range(len(wife.split())):
        if wife.split()[num][-1]=='.':
            try:
                return (wife.split()[num-1]==husband.split()[num-1])  & (wife.split()[num+1]==husband.split()[num+1])
            except:
                return False
            
        #Splitting by gender
def dead_husband(wife,husbandlist):
    #this takes a wife's name as an argument and a listas the second argument.  it returns a true or false
    result= False # assume that the wife did not die
    for num in range(len(husbandlist)):
        if findhusband(wife, husbandlist[num]):
            result = True
    return result
def dead_wife(husband,wifelist):
    result= False # assume that the wife did not die
    for num in range(len(wifelist)):
        if findhusband(wifelist[num], husband):
            result = True
    return result
            
titanicmale=  titanic[titanic['Sex']=='male'] 
titanicfemale= titanic[titanic['Sex']=='female']
testmale=  testdf[testdf['Sex']=='male'] 
testfemale= testdf[testdf['Sex']=='female']


In [52]:
livemen= titanicmale[titanicmale['Survived']==1]
deadmen= titanicmale[titanicmale['Survived']==0]
livewomen= titanicfemale[titanicfemale['Survived']==1]
deadwomen= titanicfemale[titanicfemale['Survived']==0]
deadwomennames= deadwomen.Name.values
livewomennames= livewomen.Name.values
livemennames = livemen.Name.values
deadmennames = deadmen.Name.values

In [53]:
# lets do women with dead husbands. 
titanicfemale['Dead_Husband']= titanicfemale.Name.apply(lambda x:  dead_husband(x,deadmennames))
titanicfemale['Live_Husband']= titanicfemale.Name.apply(lambda x:  dead_husband(x,livemennames))
titanicmale['Dead_Wife']=titanicmale.Name.apply(lambda x:  dead_wife(x,deadwomennames))
titanicmale['Live_Wife']=titanicmale.Name.apply(lambda x:  dead_wife(x,livewomennames))
# lets do women with dead husbands. 
testfemale['Dead_Husband']= testfemale.Name.apply(lambda x:  dead_husband(x,deadmennames))
testfemale['Live_Husband']= testfemale.Name.apply(lambda x:  dead_husband(x,livemennames))
testmale['Dead_Wife']=testmale.Name.apply(lambda x:  dead_wife(x,deadwomennames))
testmale['Live_Wife']=testmale.Name.apply(lambda x:  dead_wife(x,livewomennames))

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [54]:
testdf.loc[testfemale.index,'Dead_Husband']= testfemale['Dead_Husband'].astype(int)

testdf.loc[testfemale.index,'Live_Husband']= testfemale['Live_Husband'].astype(int)
testdf.loc[testmale.index,'Dead_Wife']= testmale['Dead_Wife'].astype(int)
testdf.loc[testmale.index,'Live_Wife']= testmale['Live_Wife'].astype(int)
titanic.loc[titanicfemale.index,'Dead_Husband']= titanicfemale['Dead_Husband'].astype(int)
titanic.loc[titanicfemale.index,'Live_Husband']= titanicfemale['Live_Husband'].astype(int)
titanic.loc[titanicmale.index,'Dead_Wife']= titanicmale['Dead_Wife'].astype(int)
titanic.loc[titanicmale.index,'Live_Wife']= titanicmale['Live_Wife'].astype(int)
titanic['Dead_Husband'].fillna(value=0, inplace=True)
titanic['Dead_Wife'].fillna(value=0, inplace=True)
testdf['Dead_Husband'].fillna(value=0, inplace=True)
testdf['Dead_Wife'].fillna(value=0, inplace=True)
titanic['Live_Husband'].fillna(value=0, inplace=True)
titanic['Live_Wife'].fillna(value=0, inplace=True)
testdf['Live_Husband'].fillna(value=0, inplace=True)
testdf['Live_Wife'].fillna(value=0, inplace=True)

In [55]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,IsMale,IsAlone,Deck,Title,Ticketnum,AgeBand,Dead_Husband,Live_Husband,Dead_Wife,Live_Wife
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,...,1,0,?,Mr.,2,2,0.0,0.0,0.0,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,...,0,0,C,Mrs.,1,3,0.0,0.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,...,0,1,?,Miss,3,3,0.0,0.0,0.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,...,0,0,C,Mrs.,1,3,1.0,0.0,0.0,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,...,1,1,?,Mr.,3,3,0.0,0.0,0.0,0.0


In [56]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,IsMale,IsAlone,Deck,Title,Ticketnum,AgeBand,Dead_Husband,Live_Husband,Dead_Wife,Live_Wife
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,...,1,0,?,Mr.,2,2,0.0,0.0,0.0,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,...,0,0,C,Mrs.,1,3,0.0,0.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,...,0,1,?,Miss,3,3,0.0,0.0,0.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,...,0,0,C,Mrs.,1,3,1.0,0.0,0.0,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,...,1,1,?,Mr.,3,3,0.0,0.0,0.0,0.0


In [57]:
def dummies2(df):
    cols = [ 'Embarked','Deck', 'Title']
    return pd.get_dummies(df[cols], columns=cols)
dummies_tf2 = FunctionTransformer(dummies2, validate=False)
def numeric_features2(df):
    return df[['Pclass', 'IsFemale',
       'AgeBand',  'Ticketnum']]
numeric_features_tf2 = FunctionTransformer(numeric_features2, validate=False)

In [58]:
survive_transformers = [('numfeat' ,numeric_features_tf2),('dummies' ,dummies_tf2)]

In [59]:
survivefu = FeatureUnion(survive_transformers)
model = AdaBoostClassifier()



In [60]:
numeric_features_tf.fit(titanic.drop('Survived', axis=1))

FunctionTransformer(accept_sparse=False,
          func=<function numeric_features at 0x7f274ef382f0>,
          inv_kw_args=None, inverse_func=None, kw_args=None, pass_y=False,
          validate=False)

In [61]:
#dummies_tf.fit(titanic.drop('Survived', axis=1))

In [62]:
survive_pipe =  Pipeline(
    [
        ('nums',survivefu),
        ('model',model)
    ]

)

In [63]:
X= titanic.drop(['Survived','PassengerId','IsMale'], axis=1)

In [64]:
X.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked', 'FamilyCount', 'IsFemale', 'IsAlone', 'Deck',
       'Title', 'Ticketnum', 'AgeBand', 'Dead_Husband', 'Live_Husband',
       'Dead_Wife', 'Live_Wife'],
      dtype='object')

In [65]:
X.isnull().any()

Pclass          False
Name            False
Sex             False
Age             False
SibSp           False
Parch           False
Ticket          False
Fare            False
Cabin           False
Embarked        False
FamilyCount     False
IsFemale        False
IsAlone         False
Deck            False
Title           False
Ticketnum       False
AgeBand         False
Dead_Husband    False
Live_Husband    False
Dead_Wife       False
Live_Wife       False
dtype: bool

In [66]:
X[X['FamilyCount'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,...,IsFemale,IsAlone,Deck,Title,Ticketnum,AgeBand,Dead_Husband,Live_Husband,Dead_Wife,Live_Wife


In [67]:
y= titanic['Survived']

In [68]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [69]:
params = {}
gs = GridSearchCV(survive_pipe, param_grid=params)
gs.fit(X, y)
print(gs.best_score_)
print(gs.best_params_)

0.801346801347
{}


In [70]:
model2 = GradientBoostingClassifier( )

In [71]:
survive_pipe2 =  Pipeline(
    [
        ('nums',survivefu),
        ('model',model2)
    ]

)

In [72]:
kparams= {
    'model__min_samples_split':range(2,3),
    'model__min_samples_leaf':range(2,3),
#    'model__min_weight_fraction_leaf':[0.0],
    'model__max_depth':range(2,3),
    'model__n_estimators':[90,100,110,120],
   'model__learning_rate':np.linspace(.02,.01,6)
}
#kparams = {}

In [73]:
params2 = {'model__learning_rate':np.linspace(.05,.25,4),'model__min_samples_split':range(2,3),'model__max_depth':range(2,10)}
gs2 = GridSearchCV(survive_pipe2, param_grid=kparams, cv=9)
gs2.fit(X_train, y_train)
print(gs2.best_score_)
print(gs2.best_params_)

0.814371257485
{'model__learning_rate': 0.02, 'model__max_depth': 2, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 110}


In [74]:
gs2.score(X_test,y_test)

0.81165919282511212

In [75]:
pred = gs2.predict(testdf.drop(['PassengerId','IsMale'], axis=1))

In [76]:
submission  = pd.DataFrame({'Survived':pred}, index=testdf.PassengerId.values)

In [77]:
submission.head()

,Survived
892,0
893,0
894,0
895,0
896,0


In [78]:
submission.to_csv('submission7265.csv', index_label='PassengerId')

In [79]:
predconf = gs2.predict(X)

In [80]:
titanic['pred']= predconf

In [81]:
confusion = titanic[(titanic['pred']!=titanic['Survived'])]

In [82]:
confusion[confusion['Survived']==1].shape

(114, 25)

In [83]:
confusion[confusion['Survived']==0].shape

(36, 25)

In [84]:
confusion[confusion['Survived']==1]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,IsAlone,Deck,Title,Ticketnum,AgeBand,Dead_Husband,Live_Husband,Dead_Wife,Live_Wife,pred
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,...,1,?,Miss,3,3,0.0,0.0,0.0,0.0,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,...,0,?,Mrs.,3,3,0.0,0.0,0.0,0.0,0
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.7000,...,0,G,Miss,9,1,0.0,0.0,0.0,0.0,0
17,18,1,2,"Williams, Mr. Charles Eugene",male,38.4,0,0,244373,13.0000,...,1,?,Mr.,2,3,0.0,0.0,0.0,0.0,0
21,22,1,2,"Beesley, Mr. Lawrence",male,34,0,0,248698,13.0000,...,1,D,Mr.,2,3,0.0,0.0,0.0,0.0,0
23,24,1,1,"Sloper, Mr. William Thompson",male,28,0,0,113788,35.5000,...,1,A,Mr.,1,3,0.0,0.0,0.0,0.0,0
25,26,1,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38,1,5,347077,31.3875,...,0,?,Mrs.,3,3,0.0,0.0,0.0,0.0,0
36,37,1,3,"Mamee, Mr. Hanna",male,22.5,0,0,2677,7.2292,...,1,?,Mr.,2,2,0.0,0.0,0.0,0.0,0
55,56,1,1,"Woolner, Mr. Hugh",male,44.2,0,0,19947,35.5000,...,1,C,Mr.,1,4,0.0,0.0,0.0,0.0,0
68,69,1,3,"Andersson, Miss. Erna Alexandra",female,17,4,2,3101281,7.9250,...,0,?,Miss,3,2,0.0,0.0,0.0,0.0,0
